In [163]:
import numpy as np
import pandas as pd

In [164]:
# Get a files for study
#a_19 = pd.read_table("study_data/MTBLS72_data/a_MTBLS19_metabolite profiling_mass spectrometry.txt")
#a_19_pos = pd.read_table("study_data/MTBLS72_data/a_pos_MTBLS19_metabolite profiling_mass spectrometry.txt")
#a_19_neg = pd.read_table("study_data/MTBLS72_data/a_neg_MTBLS19_metabolite profiling_mass spectrometry.txt")

In [241]:
# Get s files for study - these both look the same
s_72_1 = pd.read_table("study_data/MTBLS72_data/s_Plasma AD Lipidomics.txt")
s_72_2 = pd.read_table("study_data/MTBLS72_data/s_Plasma_AD_Lipidomics.txt")

In [242]:
#Get features for study
f_72_neg = pd.read_csv("study_data/reprocessed_data/IPO_aligned_MTBLS72_neg.csv", index_col=0)
f_72_pos = pd.read_csv("study_data/reprocessed_data/IPO_aligned_MTBLS72_pos.csv", index_col=0)

In [243]:
name_to_disease = s_72_1[['Sample Name', 'Factor Value[Cognitive Status]']]
name_to_disease = name_to_disease.set_index('Sample Name')
f_72_neg['drt'] = f_72_neg['rtmax'] - f_72_neg['rtmin']
f_72_pos['drt'] = f_72_pos['rtmax'] - f_72_pos['rtmin']
f_72_neg = f_72_neg.T
f_72_pos = f_72_pos.T

In [244]:
f_72_neg = f_72_neg.reset_index()
f_72_pos = f_72_pos.reset_index()

In [245]:
sample_list = name_to_disease.index.tolist()
f_list = [f_72_neg, f_72_pos]
for i,f_72 in enumerate(f_list):
    for index, row in f_72.iterrows():
        s = row['index']
        if '.mzML' in s:
            s = s[1:]
            s = s.split('.mzML')[0]
            f_72.loc[index,'Disease']= name_to_disease.loc[s,'Factor Value[Cognitive Status]']
            s = s.split('_')
            s = s[0]+'_'+s[-1]
            f_72.loc[index,'new_index'] = s
        elif 'drt' in s or 'npeaks' in s or 'pcgroup' in s:
            f_72.loc[index, 'new_index'] = s
            f_72.loc[index, 'Disease'] = 'NA'
    f_72.dropna(inplace=True, subset=['Disease'])
    f_72.set_index('new_index', inplace=True)
    f_72.drop('index', axis = 1, inplace=True)
        
        

In [246]:
f_72 = pd.concat([f_72_neg, f_72_pos], axis = 1)
disease = f_72['Disease'].iloc[:,0]
f_72.drop(inplace=True, labels=['Disease'], axis=1)
f_72 = f_72.T.reset_index(drop=True).T
f_72 = pd.concat([f_72, disease], axis = 1)

In [247]:
f_72.to_csv('study_72_processed.csv')